In [34]:


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
spark = SparkSession.builder\
	.appName("pyspark_stream_setup").getOrCreate()


In [36]:
my_schema = StructType() \
                    	.add("PassengerId", "string")\
                    	.add("Survived", "string")\
                    	.add("Pclass", "string")\
                    	.add("Name", "string")\
                    	.add("Sex", "string")\
                        .add("Age", "integer")\
                        .add("SibSp", "string")\
                        .add("Parch", "string")\
                        .add("Ticket", "string")\
                        .add("Fare", "string")\
                        .add("Cabin", "string")\
                        .add("Embarked", "string")


In [37]:
# printing the schema 

my_schema.simpleString()

'struct<PassengerId:string,Survived:string,Pclass:string,Name:string,Sex:string,Age:int,SibSp:string,Parch:string,Ticket:string,Fare:string,Cabin:string,Embarked:string>'

In [38]:
# creating df from stream 

stream_df  = spark.readStream\
                        	.option("sep", ",")\
                        	.schema(my_schema)\
                        	.csv("streams_dir/titanic.csv")

In [40]:
# making sure the df is in the stream mode  

stream_df.isStreaming

True

In [43]:
df_read = spark\
            .readStream\
            .option('header' , True)\
            .schema(my_schema)\
            .option('maxfilesperTrigger' , 1)\
            .csv('streams_dir/')

In [44]:
display(df_read)

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: int, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [47]:
df_read.writeStream\
    .format('console')\
    .trigger(processingTime="5 seconds")\
    .outputMode('update')\
    .start()

22/06/13 23:40:09 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-fd3aadd3-77b9-4340-aeaa-04371a82d7fe. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/13 23:40:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05